In [1]:
import re
import mysql.connector
import glob
from datetime import datetime

In [2]:
db_config = {
    "host" : "localhost",
    "user" : "root",
    "password" : "730203",
    "database" : "emoji"
}

In [3]:
def extract_emoji(text):
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F" 
        u"\U0001F300-\U0001F5FF" 
        u"\U0001F680-\U0001F6FF"
        u"\U0001F1E0-\U0001F1FF"
        u"\U0001F900-\U0001F9FF"
        u"\U0001FA00-\U0001FA6F"
        u"\U0001F700-\U0001F77F"
        u"\U0001F780-\U0001F7FF"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.findall(text)

In [4]:
def convert_date(date_str):
    date_obj = datetime.strptime(date_str, '%Y년 %m월 %d일')
    return date_obj.strftime('%Y-%m-%d')

In [5]:
def convert_time(time_str):
    time_obj = datetime.strptime(time_str, '%p %I:%M')
    return time_obj.strftime('%H:%M')

In [12]:
chat_data = []

for file_name in glob.glob('*.txt'):
    with open(file_name, 'r', encoding='utf-8') as file:
        lines = file.readlines()
        first_line = lines[0]
        name_end_index = first_line.find('님과 카카오톡 대화')
        participant1 = first_line[:name_end_index]
        participant1 = participant1.strip()
        participant2 = '김성연'
        
        for line in lines:
            if line.startswith('---------------'):
                match = re.search(r'\d{4}년 \d+월 \d+일', line)
                if match:
                    date = match.group(0)
                    date = convert_date(date)

            elif line.startswith('['):
                sender, time, text = line.split('] ')
                sender = sender[1:]
                
                if sender == participant1:
                    receiver = participant2
                else:
                    receiver = participant1
                    
                time = time[1:].replace('오전', 'AM').replace('오후', 'PM')
                time = convert_time(time)
                emoji_text = extract_emoji(text)
                
                for emojis in emoji_text:
                    for emoji in emojis:
                        chat_data.append((sender, receiver, (date+' '+time+":00"), emoji))

In [13]:
chat_data

[('유재혁', '김성연', '2022-12-07 22:01:00', '💌'),
 ('김성연', '유재혁', '2022-12-07 22:19:00', '🥺'),
 ('유재혁', '김성연', '2022-12-25 14:41:00', '🎄'),
 ('유재혁', '김성연', '2022-12-25 14:41:00', '🎄'),
 ('유재혁', '김성연', '2022-12-28 22:19:00', '😊'),
 ('유재혁', '김성연', '2022-12-28 22:19:00', '😊'),
 ('유재혁', '김성연', '2023-01-02 17:23:00', '😊'),
 ('유재혁', '김성연', '2023-01-02 17:23:00', '😊'),
 ('유재혁', '김성연', '2023-01-04 02:04:00', '😊'),
 ('유재혁', '김성연', '2023-02-05 21:36:00', '😝'),
 ('유재혁', '김성연', '2023-02-05 21:36:00', '😝'),
 ('김성연', '유재혁', '2023-02-08 21:52:00', '😉'),
 ('유재혁', '김성연', '2023-02-14 16:07:00', '😢'),
 ('유재혁', '김성연', '2023-02-17 00:59:00', '😊'),
 ('유재혁', '김성연', '2023-02-23 00:16:00', '🎉'),
 ('유재혁', '김성연', '2023-02-23 00:16:00', '🎉'),
 ('유재혁', '김성연', '2023-03-30 15:48:00', '😢'),
 ('유재혁', '김성연', '2023-03-30 15:48:00', '😢'),
 ('유재혁', '김성연', '2023-04-12 20:37:00', '👍'),
 ('유재혁', '김성연', '2023-04-12 20:37:00', '🏻'),
 ('김성연', '유재혁', '2023-04-19 19:49:00', '😢'),
 ('유재혁', '김성연', '2023-04-28 15:57:00', '👍'),
 ('유재혁', '

In [14]:
try:
    connection = mysql.connector.connect(**db_config)
    cursor = connection.cursor()
    
    insert_query = "INSERT INTO chat_list (sender, receiver, date, emoji) VALUES (%s, %s, %s, %s)"
    
    cursor.executemany(insert_query, chat_data)
    connection.commit()
    print(f"{cursor.rowcount} rows inserted.")
    
except mysql.connector.Error as err:
    print("Error:", err)
    
finally:
    if connection.is_connected():
        cursor.close()
        connection.close()
        print("MySQL connection closed.")

1452 rows inserted.
MySQL connection closed.
